In [12]:
import pandas as pd
import numpy as np
import os
import pickle
import sys
from sklearn.neighbors import KNeighborsClassifier  
import matplotlib.pyplot as plt
from sklearn import  preprocessing
import networkx as nx
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2
from sklearn.tree import DecisionTreeClassifier
import time
from db.database import DatabaseConnector
from dtloader.dataloader import DataLoader
from sklearn.model_selection import cross_val_score
from statsmodels.tsa.stattools import grangercausalitytests
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.utils import *
import seaborn as sns
from granger.grangercausality_reg import GrangerCausalityTest
from granger.grangercausality_cl import GrangerCausalityDiscrete
from segmentation.segmentation import *
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.tsatools import lagmat2ds
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
from utils.features_eng import *
from scipy import stats
from sklearn.model_selection import cross_val_score
from statsmodels.tsa.stattools import grangercausalitytests
from causality_utils import *
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
#connecting to the database 
dl = DataLoader()
dl.load('../sample_testing/sample2.log')
dl.extractinfo(export=False,single_file=False)

mongodb://localhost:27017/
Database connected


In [ ]:
#dl.dbconnector.query_str('sample2')

In [ ]:
pd.DataFrame(dl.dbconnector.query('CTUN_sample_testing/sample2')).head()

# Sample analysis and testing
### X Causes Z

In [13]:
x = 'CTUN_ThrOut'
y = 'RCIN_C2'
z = 'ATT_Yaw'
#5779b87af6909000298709e9
filename = '5779b87af6909000298709e9'
lag = 30
cv = 8

In [19]:
mean_acc_full_list = []
mean_acc_red_list = []
p_value_list = []

dft_x, dft_y, dataset_x, dataset_y, dataset_z, dataset_combined_xz, dataset_combined_yz, target_x, target_y = causality_cls_preprocess(x,y,z
,filename, lag,dl)

comp_y, va_y = signal_comp_va(y)
comp_z, va_z = signal_comp_va(z)
comp_x, va_x = signal_comp_va(x)

dict_cls =  test_causality(dft_x,comp_x,va_x,dataset_z,dataset_combined_xz,comp_z,va_z,target_x,cv=cv)
resdf = pd.DataFrame(dict_cls).T

print('Mean Across Full Classifiers : ',resdf['Mean-Acc-Full'].mean())
print('Mean Across Reduced Classifiers : ',resdf['Mean-Acc-Reduced'].mean())
resdf

Mean Across Full Classifiers :  0.6513534243426287
Mean Across Reduced Classifiers :  0.6428408280431233


,Mean-Acc-Full,Mean-Acc-Reduced,P-Value
nb,0.585390,0.778404,0.204168
dt,0.458184,0.447043,0.943118
knn,0.647912,0.432419,0.128449
lr,0.913928,0.913498,0.948551


In [ ]:
clf = DecisionTreeClassifier()
clf.fit(dataset_combined_xz,target_x)
(pd.Series(clf.feature_importances_, index=dataset_combined_xz.columns).plot(kind='barh')) 
print(clf.feature_importances_)

# Y Causes Z

In [ ]:
dict_cls =  test_causality(dft_y,comp_y,va_y,dataset_z,dataset_combined_yz,comp_z,va_z,target_y,cv=cv)
resdf = pd.DataFrame(dict_cls).T
print('Mean Across Full Classifiers : ',resdf['Mean-Acc-Full'].mean())
print('Mean Across Reduced Classifiers : ',resdf['Mean-Acc-Reduced'].mean())
resdf

### ---------------------------------------------------------------------------------------------------------------------------------------------------------------
# Aggergated Across Logs Files

In [10]:
datalist =os.listdir('../data')
lag = 30
cv = 8
comp_y, va_y = signal_comp_va(y)
comp_z, va_z = signal_comp_va(z)
comp_x, va_x = signal_comp_va(x)
lag_list = [25,40,55,75]

NameError: name 'signal_comp_va' is not defined

In [7]:
np.random.choice(datalist, 5,replace=False)

NameError: name 'datalist' is not defined

In [ ]:

dict_files = {} 
for i,log in enumerate(datalist):
        print('File:{} --  {} / {}'.format(log,i,len(datalist)-1))
        dft_x, dft_y, dataset_x, dataset_y, dataset_z, dataset_combined_xz, dataset_combined_yz, target_x, target_y = causality_cls_preprocess(x,y,z
        ,log[:-4], lags)
        if isinstance(dft_x,int):
            print('One of the signals not found in file.')
            continue

        #added for fruther analysis and affects performance
        sr_x = dl.dbconnector.query(comp_x+'_' + log[:-4],va_x)
        sr_z = dl.dbconnector.query(comp_z+'_' + log[:-4],va_z)

        try:
            dict_cls =  test_causality(dft_x,comp_x,va_x,dataset_z,dataset_combined_xz,comp_z,va_z,target_x,cv=cv)
        except:
            continue
        resdf = pd.DataFrame(dict_cls).T
        dict_files[log[:-4]] = { 'mean_full_acc' :resdf['Mean-Acc-Full'].mean() , 'mean_red_acc':resdf['Mean-Acc-Reduced'].mean() ,
                                'pvalue_dt':resdf.loc['dt']['P-Value'] ,'pvalue_knn':resdf.loc['knn']['P-Value']
                                'Mean '+comp_x+'_'+va_x: np.mean(sr_x),'Mean '+comp_z+'_'+va_z: np.mean(sr_z)}
        

In [ ]:
dt_seg = dict_acc_lags.copy()
save_as_pickle(dt_seg,'pickle','dt_means.pickle')

In [ ]:
dt_seg = load_pickle('pickle','dt_means.pickle')

In [ ]:
df = pd.DataFrame(dt_seg[40]).T
df.head()

In [ ]:
sig_perc_list = []
for key,val in dt_seg.items():
    print('Lags : ', key)
    df = pd.DataFrame(dt_seg[key]).T
    print('Percentage Significant ', len(df.loc[df.pvalue <= 0.05])  / len(df))
    sig_perc_list.append((len(df.loc[df.pvalue <= 0.05])*100) / len(df))
sns.barplot(x=lag_list,y=sig_perc_list)
plt.xlabel('Lags')
plt.ylabel('Percentage Siginficant')
plt.show()
print('Total File Number',len((df)))

# Significant Files

In [ ]:
df = pd.DataFrame(dt_seg[40]).T

In [ ]:
df_sig = df.loc[df.pvalue <= 0.05]#[['Mean BARO_Alt','Mean RCIN_C3']]
#.plot(kind='kde',figsize=(12,9))
df_non_sig = df.loc[df.pvalue > 0.05]

In [ ]:
comb = pd.DataFrame({'sig':df_sig['Mean BARO_Alt'],'non-sig':df_non_sig['Mean BARO_Alt']})

In [ ]:
plt.figure(figsize=(12,9))
sns.kdeplot(df_non_sig['Mean BARO_Alt'].sample(frac=0.3),label='Mean Non-Sig')
sns.kdeplot(df_sig['Mean BARO_Alt'],label='Mean Sig')
plt.title('Significant files : Mean BARO_Alt')
plt.xlabel('Mean')
plt.ylabel('Prob')

In [ ]:
df.loc[df.pvalue <= 0.05,['mean_full_acc','mean_red_acc']].plot(figsize=(12,9))
plt.title('Significant pvalues logs: Mean Acc vs logs')
plt.xlabel('logs')
plt.ylabel('Mean Acc')

In [ ]:
df.loc[df.pvalue > 0.05,['mean_full_acc','mean_red_acc']].sample(frac=0.5, random_state=1).plot(figsize=(12,9))
plt.title('Non-Significant pvalues logs: Mean Acc vs logs')
plt.xlabel('logs')
plt.ylabel('Mean Acc')

In [ ]:
plt.figure(figsize=(12,9))
sns.kdeplot(df_non_sig['Mean RCIN_C3'].sample(frac=0.5),label='Mean Non-Sig')
sns.kdeplot(df_sig['Mean RCIN_C3'],label='Mean Sig')
plt.title('Mean RCIN_C3 dist')
plt.xlabel('Mean')
plt.ylabel('Prob')